# Imports

In [258]:
import pyaudio
import wave
import numpy as np
import logging
from collections import deque
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, rfft, irfft

logging.basicConfig(level=logging.INFO)

In [259]:
c = 343.0  # Speed of sound in m/s
d = 0.1  # Distance between microphones in meters
fs = 48000  # Sampling frequency in Hz
max_shift = int(d / c * fs) + 1  # Maximum shift in samples
shift_values = np.arange(-max_shift, max_shift + 1)

# Functions

In [246]:
def naive_shift(data):
    data_0 = data[0] / np.linalg.norm(data[0])
    data_1 = data[1] / np.linalg.norm(data[1])
    scalar_products = np.array([data_0 @ np.roll(data_1, shift=i) for i in shift_values])
    shift_raw = np.argmax(scalar_products)
    shift = shift_values[shift_raw]

    rolled_values = np.roll(scalar_products, -(shift_raw-3))[:7]
    confidence = np.max(rolled_values) -np.mean(rolled_values)
    return shift, confidence

In [247]:
def gcc_phat(filtered_data):
    X = rfft(filtered_data)
    cross_corr = X[0] * np.conj(X[1]) + 1e-12*np.ones_like(X[0])
    shift_signal = irfft(cross_corr / np.abs(cross_corr))

    shift_signal_pos = shift_signal[:max_shift+1]
    shift_signal_neg = shift_signal[-max_shift:]
    argmax_pos = np.argmax(shift_signal_pos)
    maxval_pos = shift_signal_pos[argmax_pos]
    argmax_neg = np.argmax(shift_signal_neg)
    maxval_neg = shift_signal_neg[argmax_neg]
    if maxval_pos > maxval_neg:
        shift = argmax_pos
    else:
        shift = argmax_neg - max_shift

    return shift

# Audio

## Record

In [ ]:
chunk_size = 4800
dtype = pyaudio.paInt16
n_channels = 2
rate = 48000
record_seconds = 60
wave_output_filename = "output.wav"

In [ ]:
p = pyaudio.PyAudio()
stream = p.open(format=dtype,
                channels=n_channels,
                rate=rate,
                input=True,
                # input_device_index=2,
                frames_per_buffer=chunk_size)
logging.info("* recording")
frames = []
for i in range(0, int(rate / chunk_size * record_seconds)):
    raw_data = stream.read(chunk_size)
    frames.append(raw_data)
logging.info("* done recording")
stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
full_binary = b''.join(frames)
full_array = np.frombuffer(full_binary, dtype=np.int16)
channel_0 = full_array[0::n_channels]
channel_1 = full_array[1::n_channels]
channels = np.vstack((channel_0, channel_1))#, channel_2, channel_3))
channels.shape

## Plot

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(channels[0], c='blue')
plt.plot(channels[1], c='orange',alpha=0.75)
# plt.plot(channels[2], c='green')
# plt.plot(channels[3], c='red')
# plt.xlim(14680, 14720)

In [ ]:
start = 2700000
X = rfft(channels[:,start:start+48000])

fig, axs = plt.subplots(ncols=3, nrows=2, figsize =(15, 10))
axs[0,0].plot(np.abs(X[0][200:300]), c='blue', label="200")
axs[0,1].plot(np.abs(X[0][400:600]), c='blue', label="400")
axs[0,2].plot(np.abs(X[0][600:800]), c='blue', label="600")
axs[1,0].plot(np.abs(X[0][900:1000]), c='blue', label="900")
axs[1,1].plot(np.abs(X[0][1100:1300]), c='blue', label="1100")
axs[1,2].plot(np.abs(X[0][1300:1500]), c='blue', label="1300")
for ax in axs.flat:
    ax.legend()
# plt.plot(np.abs(X[0]), c='blue')
# plt.plot(np.abs(X[1]), c='orange',alpha=0.75)

In [ ]:
X = rfft(channels)

filtration_intervals = []
running_index = 0
for val in [235, 470, 705, 940, 1175, 1410]:
    filtration_intervals.append((running_index, val-40))
    running_index = val + 40
filtration_intervals.append((running_index, X.shape[1]))

for interval in filtration_intervals:
    X[:, interval[0]:interval[1]] = 0

plt.figure(figsize=(15,5))
plt.plot(np.abs(X[0]), c='blue')
plt.plot(np.abs(X[1]), c='orange',alpha=0.75)
plt.xlim(0,1500)

In [ ]:
filtered = irfft(X)

plt.figure(figsize=(15,5))
plt.plot(filtered[0], c='blue')
plt.plot(filtered[1], c='orange', alpha=0.75)

## Compute Shift

In [ ]:
scalar_products = np.array([filtered[0] @ np.roll(filtered[1], shift=i) for i in shift_values])

In [ ]:
max_shift_raw = np.argmax(scalar_products)
max_shift = shift_values[max_shift_raw]

In [ ]:
max_shift

## Save

In [ ]:
# channel = 3
# wf = wave.open(f"output_channel_{channel}.wav", 'wb')
# wf.setnchannels(1)
# wf.setsampwidth(p.get_sample_size(dtype))
# wf.setframerate(rate)
# wf.writeframes(full_binary[channel::n_channels])
# wf.close()

In [ ]:
# channel = 0
# with wave.open("output.wav") as wav:
#     nch   = wav.getnchannels()
#     depth = wav.getsampwidth()
#     wav.setpos(0)
#     sdata = wav.readframes(wav.getnframes())

#     # Extract channel data (24-bit data not supported)
#     typ = { 1: np.uint8, 2: np.uint16, 4: np.uint32 }.get(depth)
#     if not typ:
#         raise ValueError("sample width {} not supported".format(depth))
#     print ("Extracting channel {} out of {} channels, {}-bit depth".format(channel+1, nch, depth*8))
#     data = np.fromstring(sdata, dtype=typ)
#     print(data.shape)
#     ch_data = data[channel::nch]
#     print(ch_data.shape)

#     # # Save channel to a separate file
#     # outwav = wave.open(filename, 'w')
#     # outwav.setparams(wav.getparams())
#     # outwav.setnchannels(1)
#     # outwav.writeframes(ch_data.tostring())
#     # outwav.close()

# Orchestrate

In [268]:
rate = 48000
computations_frequency = 10  # per second
chunk_size = int(rate/computations_frequency)
dtype = pyaudio.paInt16
n_channels = 4
record_seconds = 100
target_frequency_raw = 500
target_frequency = target_frequency_raw / computations_frequency

In [269]:
def box_filter(X, filtration_values, filtration_width):
    X_filtered = X.copy()
    filtration_intervals = []
    running_index = 0
    for val in filtration_values:
        filtration_intervals.append((running_index, val-filtration_width))
        running_index = val + filtration_width
    filtration_intervals.append((running_index, X.shape[1]))

    for interval in filtration_intervals:
        X_filtered[:, interval[0]:interval[1]] = 0

    return X_filtered

In [272]:
p = pyaudio.PyAudio()
stream = p.open(format=dtype,
                channels=n_channels,
                rate=rate,
                input=True,
                # input_device_index=2,
                frames_per_buffer=chunk_size)
logging.info("* recording")

frames = []
shift_history = deque(maxlen=10)
for i in range(0, int(rate / chunk_size * record_seconds)):
    raw_data = stream.read(chunk_size)
    # frames.append(raw_data)
    data = np.frombuffer(raw_data, dtype=np.int16)
    channel_0 = data[0::n_channels]
    channel_1 = data[1::n_channels]
    channels = np.vstack((channel_0, channel_1))#, channel_2, channel_3))

    # filter
    X = rfft(channels)
    filtered_X = box_filter(X, filtration_values=[235, 470, 705, 940, 1175, 1410], filtration_width=40)
    filtered_data = irfft(filtered_X, n=channels.shape[1])
    
    # compute shift
    shift, confidence = naive_shift(filtered_data)
    shift_history.append(shift)
    if i % 2 == 0:
        logging.info(int(np.round(np.mean(shift_history))))

    # X = rfft(channels)
    # filtration_values = [235, 470, 705, 940, 1175, 1410]
    # confidences = []
    # shifts = []
    # for val in [235, 470, 705, 940, 1175, 1410]:
    #     filtered_X = box_filter(X, filtration_values=[val], filtration_width=40)
    #     filtered_data = irfft(filtered_X, n=channels.shape[1])
    #     shift, confidence = naive_shift(filtered_data)
    #     confidences.append(confidence)
    #     shifts.append(shift)
    # ponderated_shift = int(np.round(np.average(shifts, weights=confidences)))
    # # logging.info(confidences)
    # logging.info(ponderated_shift)

logging.info("* done recording")
stream.stop_stream()
stream.close()
p.terminate()

INFO:root:* recording
INFO:root:-12
INFO:root:-5
INFO:root:-4
INFO:root:-2
INFO:root:-1
INFO:root:2
INFO:root:4
INFO:root:5
INFO:root:6
INFO:root:6
INFO:root:6
INFO:root:5
INFO:root:5
INFO:root:4
INFO:root:4
INFO:root:3
INFO:root:4
INFO:root:2
INFO:root:-1
INFO:root:-1
INFO:root:-4
INFO:root:-6
INFO:root:-7
INFO:root:-6
INFO:root:-6
INFO:root:-2
INFO:root:-4
INFO:root:-2
INFO:root:-2
INFO:root:-2
INFO:root:-4
INFO:root:-1
INFO:root:-1
INFO:root:0
INFO:root:-2
INFO:root:-2
INFO:root:-2
INFO:root:-2
INFO:root:-1
INFO:root:-2
INFO:root:1
INFO:root:1
INFO:root:1
INFO:root:0
INFO:root:3
INFO:root:2
INFO:root:1
INFO:root:2
INFO:root:3
INFO:root:2
INFO:root:2
INFO:root:3
INFO:root:2
INFO:root:2
INFO:root:2
INFO:root:2
INFO:root:1
INFO:root:1
INFO:root:2
INFO:root:2
INFO:root:0
INFO:root:-1
INFO:root:2
INFO:root:0
INFO:root:2
INFO:root:1
INFO:root:4
INFO:root:1
INFO:root:2
INFO:root:1
INFO:root:2
INFO:root:1
INFO:root:1
INFO:root:0
INFO:root:0
INFO:root:0
INFO:root:-1
INFO:root:-1
INFO:root:0


KeyboardInterrupt: 